https://github.com/alirezadir/Machine-Learning-Interviews/blob/main/src/MLC/notebooks/decision_tree.ipynb

### 동작
- 불순도를 최소화 시키는 특징으로 데이터 셋 분할
    - _best_split(X,y) -> best_idx, best_thr
    - _gini
- 데이터 셋 특정 크기 이하(또는 label 종류 수)되면 정지
    - max_depth로 제한
- 그 때 평균값으로 회귀/분류 추론?
    - Node class에 담기는 정보
        - self.predicted_class = predicted_class
        - self.feature_index = 0
        - self.threshold = 0.0 
        - self.left = None
        - self.right = None
        - is_leaf_node(self)
    - leaf 도달할 때까지 노드의 정보로 진행 후, 노드의 레이블 반환

### 특징
- 장점
  - 결정 트리는 해석 및 시각화가 용이
  - 수치 데이터와 범주형 데이터를 모두 처리 가능
  - 결측값을 처리할 수 있다
- 단점 
  - 트리가 너무 복잡하거나 데이터에 노이즈나 이상치가 있는 경우 과적합 발생 가능
  - 결정 트리는 매우 불균형한 데이터셋이나 많은 관련 없는 특징이 있는 데이터셋에서는 성능이 좋지 않을 수 있음
  - 특징과 출력 간의 관계가 매우 비선형적이거나 복잡한 작업에는 적합하지 않을 수 있음
- 개선  
  - 가지치기(pruning), 앙상블 방법(ensemble methods), 정규화(regularization) 등 일반화 성능을 향상

In [1]:
import numpy as np

class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0.0
        self.threshold = 0.0
        self.left = None
        self.right = None
        
    def is_leaf_node(self):
        return self.left is None and self.right is None
    
        
class DecisionTree:
    def __init__(self,max_depth=None):
        self.max_depth= max_depth
    
    def _gini(self,y):
        n_samples= len(y)
        gini= 1- np.sum([ sum(np.where(y==c, 1, 0))/n_samples ** 2 for c in range(len(self.n_classes))])
        return gini        
    
    def predict(self, X):
        return [self._predict(inputs) for inputs in X]
    
    def train(self, x,  y):
        self.n_classes = np.unique(y)
        self.n_features= x.shape[1]
        self.root= self._make_tree(x, y)
    
    def _make_tree(self, x, y, depth):
        if depth >= self.max_depth:
            return None
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        # 가장 수가 많은 클래스가 해당 노드의 클래스가 된다.
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)
        
        node.feature_index, node.threshold = self._best_split_feature_idx_and_thr(x, y)
        left_indices= np.where(x[node.feature_index]<= node.threshold)
        right_indices=  np.where(x[node.feature_index] > node.threshold)
        node.left= self._make_tree(x[left_indices], y[left_indices], depth+1)
        node.left= self._make_tree(x[right_indices], y[right_indices], depth+1)
        return node
        
    def _best_split_feature_idx_and_thr(self, x, y):
        best_idx, best_thr, best_gini= None, None, self._gini(y)
        total= len(y)
        
        for idx in range(self.n_features):
            # 샘플들의 idx 번째 특성만 추출
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            # DL개, DR개 나눠서 각 집합 불순도합이 얼마나 작아지는 지 본다
            # 각 샘플의 idx번째 특성 값 중 제일 불순도 개선이 큰 값이 threshold가 된다.
            for thr_idx, thr in enumerate(thresholds):
                if i > 0 and thresholds[i] == thresholds[i - 1]:
                    continue
                dl, dr = x[:thr_idx], x[thr_idx+1:]
                ginil, ginir = self._gini(y[:thr_idx]), self._gini(y[thr_idx+1:])
                gini= len(dl)/total * ginil + len(dr)/total*ginir
                if best_gini > gini:
                    best_idx, best_thr, best_gini = idx, thr, gini
        return best_idx, best_thr
            
    def _predict(self, inputs):
        node = self.root
        # node가 leaf일 때까지 진행 후 리프의 클래스 반환
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

# 불순도
### 범주형
- Information Gain
    - 이 값이 클수록 해당 특징이 데이터를 잘 분할한다
    - 데이터셋 ( D )에서 특징 ( A )에 대한 정보 이득 
    - IG(D, A) = Entropy(D) - ∑v∈Values(A)|Dv||D| · Entropy(Dv)
- 지니 불순도(Gini Impurity)
    - 값이 0에 가까울수록 노드에 있는 샘플들이 동일한 클래스에 속할 확률이 높다
    - Gini(D) = 1 - sum_{i=1}^{C} p_i^2
    - ( D )는 데이터셋, ( C )는 클래스의 수, ( p_i )는 클래스 ( i )에 속하는 샘플의 비율
- 엔트로피(Entropy)
    - 엔트로피 값이 낮을수록 노드가 순수하다
    - Entropy(D) = - sum_{i=1}^{C} p_i*log_2(p_i)
    - ( D )는 데이터셋, ( C )는 클래스의 수, ( p_i )는 클래스 ( i )에 속하는 샘플의 비율
### 회귀
- 평균 제곱 오차(MSE) 감소
    - 예측 값과 실제 값 간의 차이를 제곱하여 평균한 값
    - MSE(D) = sum(y_i-y_pred)^2/N
    - ∆MSE = MSE(D) −( |DL|/|D|*MSE(DL) + |DR|/|D|*MSE(DR))
- 분산(Variance)
    - 회귀 트리에서 분산이 낮을수록 노드가 순수하다
    - Variance(D) = sum(y_i - bar{y})^2 / N
    - ∆Variance = Variance(D) −( |DL|/|D|*Variance(DL) + |DR|/|D|*Variance(DR))


In [2]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        
    def fit(self, X, y):
        self.n_classes_ = len(np.unique(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)
        
    def predict(self, X):
        return [self._predict(inputs) for inputs in X]
        
    def _gini(self, y):
        _, counts = np.unique(y, return_counts=True)
        impurity = 1 - np.sum([(count / len(y)) ** 2 for count in counts])
        return impurity
        
    def _best_split(self, X, y):
        m = y.size
        if m <= 1:
            return None, None
        
        num_parent = [np.sum(y == c) for c in range(self.n_classes_)]
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None
        
        for idx in range(self.n_features_):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()
            for i in range(1, m):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes_)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_)
                )
                gini = (i * gini_left + (m - i) * gini_right) / m
                if thresholds[i] == thresholds[i - 1]:
                    continue
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        
        return best_idx, best_thr
        
    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)
        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node
        
    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class
    
class Node:
    def __init__(self, *, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0.0 
        self.left = None
        self.right = None

    def is_leaf_node(self):
        return self.left is None and self.right is None